In [4]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.struct_pb2 import Struct, Value
import os

Kode di bawah menetapkan nama pipeline dan path untuk directory dari semua file untuk kebutuhan pipeline

In [5]:
PIPELINE_NAME = "red-wine-pipeline"
SCHEMA_PIPELINE_NAME = "red-wine-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

Import dan cek isi dari datset. Dataset merupakan data mengenai beberapa hal yang dapat mempengaruhi kualitas anggur merah. Pipeline ini akan menggunakan data yang tersedia untuk memprediksi kualitas berdasarkan faktor-faktor yang ada.

In [6]:
import pandas as pd
df = pd.read_csv(r'C:\Users\bisto\tfx-dicoding\misaelricardo-submission\data\winequality-red-comma.csv')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


Menetapkan lokasi dataset yang akan digunakan pada pipeline

In [7]:
DATA_ROOT = 'data'

In [8]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

Pada kode di bawah, pipeline akan menggunakan ExampleGen untuk mengambil data dan membaginya ke dalam train-eval split dengan rasio 9:1

In [9]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=9),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=1)
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [10]:
interactive_context.run(example_gen)

c:\Users\bisto\tfx-dicoding\mlops\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 234
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

Melalui StatisticsGen, kita dapat melihat beberapa informasi mengenai data seperti rata-rata, persebaran, dan lain-lain. 

In [11]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 235
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [12]:
interactive_context.show(statistics_gen.outputs["statistics"])

Melalui data yang dicek oleh StatisticsGen, SchemaGen mengecek skema data, seperti tipe data dan apa saja yang diperlukan untuk diproses.

In [13]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 236
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [14]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'alcohol',FLOAT,required,,-
'chlorides',FLOAT,required,,-
'citric acid',FLOAT,required,,-
'density',FLOAT,required,,-
'fixed acidity',FLOAT,required,,-
'free sulfur dioxide',FLOAT,required,,-
'pH',FLOAT,required,,-
'quality',INT,required,,-
'residual sugar',FLOAT,required,,-


Selanjutnya, dengan example validator, pipeline akan mengecek apakah ada anomali atau data yang dapat mempengaruhi performa model.

In [15]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 237
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [16]:
interactive_context.show(example_validator.outputs['anomalies'])

Beberapa kode di bawah adalah kode transform atau processing data untuk dilanjutkan ke tahap berikutnya. Kode transform akan mengubah nama setiap kolom yang ada.

In [17]:
TRANSFORM_MODULE_FILE = "redwinequality_transform.py"

In [18]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft
def _transformed_name(key):
    return key.replace(' ', '_').lower() + '_xf'

def preprocessing_fn(inputs):
    """Preprocess input features."""
    outputs = {}

    # Normalize numerical features
    for feature_name in ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
                         'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
                         'pH', 'sulphates', 'alcohol']:
        outputs[_transformed_name(feature_name)] = inputs[feature_name]

    # Leave 'quality' as it is (treat it as a categorical feature)
    outputs[_transformed_name('quality')] = inputs['quality']

    return outputs

Overwriting redwinequality_transform.py


In [19]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\red-wine-pipeline\Transform\transform_graph\238\.temp_path\tftransform_tmp\c5bfcf6a83b2424e934746e0003ca4ec\assets


INFO:tensorflow:Assets written to: pipelines\red-wine-pipeline\Transform\transform_graph\238\.temp_path\tftransform_tmp\c5bfcf6a83b2424e934746e0003ca4ec\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 238
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

Kode di bawah merupakan kode trainer, kode tersebut berisi semua hal yang dilakukan untuk kebutuhan melatih model memprediksi kualitas anggur merah yang ada.

In [20]:
TRAINER_MODULE_FILE = "redwinequality_trainer.py"

In [21]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
from tfx.components.trainer.fn_args_utils import FnArgs
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import numpy as np

LABEL_KEY ='quality'

def _transformed_name(key):
    return key.replace(' ', '_').lower() + '_xf'

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs=None,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = _transformed_name(LABEL_KEY))
    return dataset.repeat()

def model_builder():
    """Build machine learning model"""
    input_layers = [
        tf.keras.Input(shape=(1,), name=_transformed_name(f), dtype=tf.float32)
        for f in [
            'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
            'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
            'pH', 'sulphates', 'alcohol'
        ]
    ]
    
    concatenated_features = tf.keras.layers.concatenate(input_layers)
    x = layers.Dense(128, activation='relu')(concatenated_features)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Dense(64, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Dense(64, activation='relu')(x)
    x = layers.BatchNormalization()(x)

    outputs = layers.Dense(1)(x)
    
    # Define the model
    model = tf.keras.Model(inputs=input_layers, outputs=outputs)
    
    model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=[tf.keras.losses.MeanSquaredError()]
)
    
    # print(model)
    model.summary()
    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # Get predictions using the transformed features
        return model(transformed_features)
    
        # Sanitize input names for SavedModel signature
        sanitized_input_names = [f.replace(' ', '_').lower() + '_xf' for f in feature_spec.keys()]
    
        serving_default_signature = serve_tf_examples_fn.get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name='examples'
            )
        )
        serving_default_signature._input_signature[0].name = 'examples'  # Rename input tensor
    
        # Assign sanitized names to input tensors
        for i, input_tensor in enumerate(serving_default_signature.inputs):
            input_tensor._name = sanitized_input_names[i]  # Assign sanitized name to input tensor

    return serve_tf_examples_fn




def run_fn(fn_args: FnArgs) -> None:
    
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_loss', mode='max', verbose=1, save_best_only=True)
    
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)

    model = model_builder()

    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000, 
            validation_steps= 1000,
            epochs=10)
            
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(
                shape=[None],
                dtype=tf.string,
                name='examples'
            )
        )
    }
    
    # Save the model with signatures
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting redwinequality_trainer.py


In [22]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=TRAINER_MODULE_FILE,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=100)
)

In [23]:
interactive_context.run(trainer)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 fixed_acidity_xf (InputLayer)  [(None, 1)]          0           []                               
                                                                                                  
 volatile_acidity_xf (InputLaye  [(None, 1)]         0           []                               
 r)                                                                                               
                                                                                                  
 citric_acid_xf (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 residual_sugar_xf (InputLayer)  [(None, 1)]         0           []                           

INFO:tensorflow:Assets written to: pipelines\red-wine-pipeline\Trainer\model\239\Format-Serving\assets


1000/1000 [==============================] - 8s 7ms/step - loss: 4.1293 - mean_squared_error: 4.1252 - val_loss: 0.3628 - val_mean_squared_error: 0.3634
Epoch 2/10
 990/1000 [============================>.] - ETA: 0s - loss: 0.4519 - mean_squared_error: 0.4518
Epoch 2: val_loss did not improve from 0.36283
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4519 - mean_squared_error: 0.4518 - val_loss: 0.3558 - val_mean_squared_error: 0.3565
Epoch 3/10
 994/1000 [============================>.] - ETA: 0s - loss: 0.4210 - mean_squared_error: 0.4210
Epoch 3: val_loss did not improve from 0.36283
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4207 - mean_squared_error: 0.4207 - val_loss: 0.3427 - val_mean_squared_error: 0.3434
Epoch 4/10
 989/1000 [============================>.] - ETA: 0s - loss: 0.4021 - mean_squared_error: 0.4021
Epoch 4: val_loss improved from 0.36283 to 0.39737, saving model to pipelines\red-wine-pipeline\Trainer\model\239\Format-Se

INFO:tensorflow:Assets written to: pipelines\red-wine-pipeline\Trainer\model\239\Format-Serving\assets


1000/1000 [==============================] - 7s 7ms/step - loss: 0.4020 - mean_squared_error: 0.4020 - val_loss: 0.3974 - val_mean_squared_error: 0.3960
Epoch 5/10
 989/1000 [============================>.] - ETA: 0s - loss: 0.3895 - mean_squared_error: 0.3895
Epoch 5: val_loss did not improve from 0.39737
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3897 - mean_squared_error: 0.3898 - val_loss: 0.3703 - val_mean_squared_error: 0.3696
Epoch 6/10
 987/1000 [============================>.] - ETA: 0s - loss: 0.3753 - mean_squared_error: 0.3753
Epoch 6: val_loss did not improve from 0.39737
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3753 - mean_squared_error: 0.3753 - val_loss: 0.3643 - val_mean_squared_error: 0.3643
Epoch 7/10
1000/1000 [==============================] - ETA: 0s - loss: 0.3646 - mean_squared_error: 0.3646
Epoch 7: val_loss improved from 0.39737 to 0.39788, saving model to pipelines\red-wine-pipeline\Trainer\model\239\Format-Se

...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\red-wine-pipeline\Trainer\model\239\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\red-wine-pipeline\Trainer\model\239\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 239
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

Kode dibawah merupakan kode tuner. Tuner akan menentukan parameter baru agar model dapat memperoleh hasil yang lebih baik.

In [24]:
TUNER_MODULE_FILE = "redwinequality_tuner.py"

In [25]:
%%writefile {TUNER_MODULE_FILE}
from typing import NamedTuple, Dict, Any, Text
from tfx.components.trainer.fn_args_utils import FnArgs
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from keras_tuner.engine import base_tuner
from kerastuner.tuners import Hyperband
from keras_tuner.engine.base_tuner import BaseTuner
import tensorflow_transform as tft
import tensorflow as tf

TunerFnResult = NamedTuple('TunerFnResult', [('tuner', BaseTuner), ('fit_kwargs', Dict[Text, Any])])

LABEL_KEY='quality'

def _transformed_name(key):
    return key.replace(' ', '_').lower() + '_xf'
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

# Define input function to read CSV files
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs=None,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = _transformed_name(LABEL_KEY))
    return dataset.repeat()

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    """Build the tuner using the KerasTuner API.

    Args:
        fn_args: Holds args used to tune models as name/value pairs.

    Returns:
        A TunerFnResult that contains the tuner and fit_kwargs.
    """
    # Load the transformed data
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_set = input_fn(fn_args.train_files[0], tf_transform_output)
    val_set = input_fn(fn_args.eval_files[0], tf_transform_output)

    # Define the model builder function
    def model_builder(hp):
        """Build machine learning model with tunable hyperparameters"""
        # Define hyperparameters
        hp_units = hp.Int('units', min_value=128, max_value=512, step=32)
        hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])

        input_layers = [
            tf.keras.Input(shape=(1,), name=_transformed_name(f), dtype=tf.float32)
            for f in [
                'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
                'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
                'pH', 'sulphates', 'alcohol'
            ]
        ]
        
        concatenated_features = tf.keras.layers.concatenate(input_layers)
        x = concatenated_features
        x = layers.Dense(hp_units, activation='relu')(concatenated_features)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)

        x = layers.Dense(hp_units//2, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)

        x = layers.Dense(hp_units//2, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        # Output layer with softmax activation for multi-class classification
        outputs = layers.Dense(1)(x)
        
        # Define the model
        model = tf.keras.Model(inputs=input_layers, outputs=outputs)
        
        model.compile(
            loss=tf.keras.losses.MeanSquaredError(),
            optimizer=tf.keras.optimizers.Adam(hp_learning_rate),
            metrics=[tf.keras.losses.MeanSquaredError()]
        )
        
        return model

    # Define the hyperband tuner
    tuner = Hyperband(model_builder,
                         objective='val_mean_squared_error',
                         max_epochs=10,
                         factor=3,
                         directory=fn_args.working_dir,
                         project_name='kt_hyperband')

    # Set fit arguments for the tuner
    early_stopping = EarlyStopping(monitor='val_mean_squared_error',  mode='max', min_delta=0.001, patience=5, verbose=1)

    fit_kwargs = {
        "callbacks": [early_stopping],
        'x': train_set,
        'validation_data': val_set,
        'steps_per_epoch': fn_args.train_steps,
        'validation_steps': fn_args.eval_steps
    }

    return TunerFnResult(tuner=tuner, fit_kwargs=fit_kwargs)

Overwriting redwinequality_tuner.py


In [26]:
tuner = Tuner(
    module_file=TUNER_MODULE_FILE,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=500),  # Adjust as needed
    eval_args=trainer_pb2.EvalArgs(num_steps=100)  # Adjust as needed
)

# Run the TFX pipeline with the Tuner component
interactive_context.run(tuner)

Trial 30 Complete [00h 00m 18s]
val_mean_squared_error: 0.37866663932800293

Best val_mean_squared_error So Far: 0.34382325410842896
Total elapsed time: 00h 04m 24s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines\red-wine-pipeline\.temp\240\kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
units: 384
learning_rate: 0.0001
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.34382325410842896
Trial summary
Hyperparameters:
units: 192
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0014
Score: 0.34410104155540466
Trial summary
Hyperparameters:
units: 352
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.34582123160362244
Trial summary
Hyperparameters:
units: 384
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.35650235414505005
Trial summary
Hyperparameters:
units: 320
learning_rate: 0.001
tuner/epochs: 4
tuner/initial_epoch: 2
tuner/bracket: 2
tuner/round: 1
tuner/trial_id: 0004
Score: 0.36098712682724
Trial summary
Hyperparameters:
units: 448
learning_rat

ExecutionResult(
    component_id: Tuner
    execution_id: 240
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

Selanjutnya trainer akan dilakukan lagi dengan parameter yang disediakan oleh tuner.

In [27]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=TRAINER_MODULE_FILE,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=100)
)

interactive_context.run(trainer)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 fixed_acidity_xf (InputLayer)  [(None, 1)]          0           []                               
                                                                                                  
 volatile_acidity_xf (InputLaye  [(None, 1)]         0           []                               
 r)                                                                                               
                                                                                                  
 citric_acid_xf (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 residual_sugar_xf (InputLayer)  [(None, 1)]         0           []                         

INFO:tensorflow:Assets written to: pipelines\red-wine-pipeline\Trainer\model\241\Format-Serving\assets


1000/1000 [==============================] - 7s 7ms/step - loss: 4.0804 - mean_squared_error: 4.0763 - val_loss: 0.3558 - val_mean_squared_error: 0.3547
Epoch 2/10
 990/1000 [============================>.] - ETA: 0s - loss: 0.4477 - mean_squared_error: 0.4477
Epoch 2: val_loss did not improve from 0.35575
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4475 - mean_squared_error: 0.4475 - val_loss: 0.3422 - val_mean_squared_error: 0.3414
Epoch 3/10
 996/1000 [============================>.] - ETA: 0s - loss: 0.4181 - mean_squared_error: 0.4181
Epoch 3: val_loss did not improve from 0.35575
1000/1000 [==============================] - 5s 5ms/step - loss: 0.4177 - mean_squared_error: 0.4177 - val_loss: 0.3525 - val_mean_squared_error: 0.3531
Epoch 4/10
 987/1000 [============================>.] - ETA: 0s - loss: 0.3960 - mean_squared_error: 0.3961
Epoch 4: val_loss did not improve from 0.35575
1000/1000 [==============================] - 5s 5ms/step - loss: 0.3958 - me

INFO:tensorflow:Assets written to: pipelines\red-wine-pipeline\Trainer\model\241\Format-Serving\assets


1000/1000 [==============================] - 6s 6ms/step - loss: 0.3854 - mean_squared_error: 0.3855 - val_loss: 0.3609 - val_mean_squared_error: 0.3606
Epoch 6/10
 985/1000 [============================>.] - ETA: 0s - loss: 0.3726 - mean_squared_error: 0.3727
Epoch 6: val_loss improved from 0.36085 to 0.36243, saving model to pipelines\red-wine-pipeline\Trainer\model\241\Format-Serving
INFO:tensorflow:Assets written to: pipelines\red-wine-pipeline\Trainer\model\241\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\red-wine-pipeline\Trainer\model\241\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\red-wine-pipeline\Trainer\model\241\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 241
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

Setelah melakukan training model dengan parameter yang tersedia, pipeline akan mengecek apakah performa model sudah sesuai standar yang kita terapkan.

In [35]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 246
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

In [33]:
import tensorflow_model_analysis as tfma

mae_threshold = tfma.MetricThreshold(
    value_threshold=tfma.GenericValueThreshold(
        upper_bound={'value': 0.6}  # Example threshold value
    )
)

mse_threshold = tfma.MetricThreshold(
    value_threshold=tfma.GenericValueThreshold(
        upper_bound={'value': 0.6}  # Example threshold value
    )
)

rmse_threshold = tfma.MetricThreshold(
    value_threshold=tfma.GenericValueThreshold(
        upper_bound={'value': 0.8}  # Example threshold value
    )
)
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='quality')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='MeanAbsoluteError',
                              threshold=mae_threshold),
            tfma.MetricConfig(class_name='MeanSquaredError',
                              threshold=mse_threshold),
            tfma.MetricConfig(class_name='RootMeanSquaredError',
                              threshold=rmse_threshold),
        ])
    ]
)

Apabila performa model berhasil mencapai performa yang diinginkan, evaluator akan menganggap model kita sebagai "blessed" dan disetujui untuk lanjut ke tahap selanjutnya.

In [34]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 245
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [31]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'mean_absolu…

Apabila model sudah disetujui oleh evaluator, model akan melalui pusher dan tersimpan di directory pipeline.

In [36]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
 
pusher = Pusher(
model=model_resolver.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/red-wine-quality-model'))
 
)
 
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 247
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

Setelah membuat Dockerimage untuk kebutuhan model, kita dapat menguji dengan cara mengaksesnya dengan kode berikut

In [38]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/red-wine-quality-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1718790092'}]}
